# Introduction
While dealing with a multi-class classification problem, logistic regression OvR can be applied only if the classes are linearly separable. hence polynomial regression should be used while dealing with a non-linear separation boundary, but the problem with this is that, feature engineering is difficult, as in, it is hard to guess which features are to be used for model building, what other relavant features can be created from existing features, etc.

K-Nearest Neighbors (KNN) is a supervised machine learning algorithm widely used popularly for classification tasks, and regression tasks as well. It works by classifying a data point based on the labels of its nearest neighbors in the training data.

KNN assumes that similar data points tend to have similar labels. To classify a new data point, KNN finds the K closest data points (neighbors) in the training set based on distance metric (Euclidean, Manhattan, Cosine, etc). The majority vote (for classification), or average value (for regression) of these K neighbors is used to predict the label or value of the new data point.

# KNN Algorithm
1. Data preparation:
    - The data is prepared by ensuring that it is in a suitable format for distance calculation (usually numerical features).
    - Feature scaling might be necessary if features have different ranges.
2. Distance calculation: A distance metric is chosen to calculate the distance between the new data point, and each data point in the training set.
3. Find the nearest neighbors: Based on the calculated distances, the K closest data points (neighbors) to the new data point are identified. The value of K is a hyperparameter that needs to be tuned.
4. Prediction:
    - For classification: The most frequent class label among the K nearest neighbors is assigned to new data point (majority vote).
    - For regression: The average value of the target variable for the K nearest neighbors is used to predict the value for the new data point.

Alternatively,
1. For an unknown data point $x_i$, the distance between it and every training data point in the dataset. The choice of the distance metric depends on the number of dimensions.
2. After calculating the distances for all the training points, they are sorted in ascending order (lowest to highest) based on the distance from the new data point $x_i$. This ensures that the data points closest to $x_i$ are at the beginning of the sorted list.
3. The value of K, a hyperparameter, determines the number of nearest neighbors to consider for prediction. The K hyperparameter is curcial, and can be tuned based on the specific problem, and dataset. After sorting, the top K data points are chosen as the nearest neighbors for the new data point.
4. The value of K chosen should be such that it should not be a multiple of the number of classes. This is done to prevent a scenario where the unknown data point is at equidistant from 2 sets of data points. But in case where, after hyprparameter tuning, the best value IS a number that is a multiple of the number of classes, then in that case, either pick random between the classes, or pick the one that is the closest. Meaning, sum up the distances of all the points present in a particular class, do the same for all the classes. Now pick the class which has the lowest sum of distance.